<a href="https://colab.research.google.com/github/ops865ops865/main-tibame20251203/blob/main/tibame20260112.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 成品1
import urllib.request as req
import bs4 as bs
import os

# M.....html/meta.json
def get_page_data(url, dirname="."):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1766978298.A.90C.html"
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)
    # print(html)

    uid_text = "-"
    board_text = "-"
    title_text = "-"
    date_time_text = "-"
    # "uid" "type":-1/0/1(需/->/推) "text"  "ipfatetime"
    pushes = []

    metas = html.find_all("span", {"class":"article-meta-value"})
    if len(metas) == 4:
        uid, board, title, date_time = metas
        uid_text = uid.get_text().strip()
        board_text = board.get_text().strip()
        title_text = title.get_text().strip()
        date_time_text = date_time.get_text().strip()

    pushes_meta = html.find_all("div", {"class":"push"})
    for p in pushes_meta:
        p_tag, p_uid, p_text, p_ipdatetime = p.find_all("span")
        p_tag_text = p_tag.get_text().strip()
        p_uid_text = p_uid.get_text().strip()
        p_text_text = p_text.get_text().strip().replace(": ", "", 1)
        p_ipdatetime_text = p_ipdatetime.get_text().strip()

        p_tag_conv = {
            "推":1,
            "→":0,
            "噓":-1
        }
        p_tag_text = p_tag_conv[p_tag_text]

        p_dict = {
            "tag":p_tag_text,
            "uid":p_uid_text,
            "text":p_text_text,
            "ipdatetime":p_ipdatetime_text
        }
        pushes.append(p_dict)

    # extract
    main_content = html.find("div", {"id":"main-content"})
    metas = main_content.find_all("div", {"class":"article-metaline"})
    for m in metas:
        m.extract()
    metas = main_content.find_all("div", {"class":"article-metaline-right"})
    for m in metas:
        m.extract()
    metas = main_content.find_all("div", {"class":"push"})
    for m in metas:
        m.extract()
    metas = main_content.find_all("span", {"class":"f2"})
    for m in metas:
        m_text = m.get_text()
        if "發信站:" in m_text:
            m.extract()
        if "文章網址:" in m_text:
            m.extract()

    main_content_text = main_content.get_text()

    data = {
        "uid":uid_text,
        "board":board_text,
        "title":title_text,
        "datetime":date_time_text,
        "content":main_content_text,
        "pushes":pushes
    }

    if not os.path.exists(dirname):
        os.makedirs(dirname)
    print("*********"*10)
    fp = dirname + "/meta.json"
    print("fp="+fp)
    f = open(fp, "w", encoding="utf-8")
    json.dump(data, f, ensure_ascii=False, indent=4)
    f.close()
    print("*********"*10)
    return data

# 成品2
# .: 這層 ..: 上一層
def download_img(url, dirname="."):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    # url = "https://i.imgur.com/opGiGRV.jpeg"
    h = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"
    }
    r = req.Request(url, headers=h)
    resp = req.urlopen(r)
    content = resp.read()
    # 1. 純文字: txt 2. 非純文字
    # r/w+encoding      rb/wb
    fp = dirname + "/" + url.split("/")[-1]
    # print(fp)
    f = open(fp, "wb")
    f.write(content)
    f.close()

# demo
# download_img("https://i.imgur.com/opGiGRV.jpeg", "aa")



In [ ]:
def get_page_imgs(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1767140595.A.150.html"
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)

    save_dir = url.split("/")[-1]
    get_page_data(url, save_dir)

    links = html.find_all("a")
    allow_subnames = {"jpg", "png", "gif", "jpeg", "mp4"}
    for l in links:
        href = l["href"]
        sub = href.split(".")[-1]
        if sub.lower() in allow_subnames:
            print(href)
            download_img(href, save_dir)

In [ ]:
# 作業
# 1. 把抓一篇定義函式
# 2. 你要抓十頁

#[公告]要pass掉，因為會有問題
# index4004
#(本文已被刪除)裡面沒有a，title裡找不到a，== None

import urllib.request as req
import bs4 as bs
import os
import json

def get_page_data(url, dirname="."):
    # url = "https://www.ptt.cc/bbs/Beauty/index4000.html"
    # url = "https://www.ptt.cc/bbs/Beauty/index" + page + ".html"
    # print(url)
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)
    # print(html)

    metas = html.find_all("div", {"class":"r-ent"})
    # metas
    for m in metas:
        if m.find("a") == None:
            pass
        else:
            title_href = "https://www.ptt.cc" + m.find("a")["href"]
            title_text = m.find("a").get_text().strip()
            author_text = m.find("div",{"class":"author"}).get_text().strip()
            date_text = m.find("div",{"class":"date"}).get_text().strip()
            if "[公告]" in title_text:
                pass
            else:
                print(title_href)
                print(title_text)
                print(author_text)
                print(date_text)
                get_page_data(title_href)
                get_page_imgs(title_href)
                print("-"*20)
                # page = str(4004-i)

In [ ]:

import urllib.request as req
import bs4 as bs
url = "https://www.ptt.cc/bbs/Beauty/index4001.html"
resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)
i=0
titles = html.find_all("div", {"class":"title"})
for t in titles:
    if i>1:
        pass
    else:
        t_link = t.find("a")
        if not t_link == None:
            t_link_href = "https://www.ptt.cc" + t_link["href"]
            t_link_text = t_link.get_text()
            print(t_link_href)
            print(t_link_text)
            if not "公告" in t_link_text:
                get_page_imgs(t_link_href)
                get_page_data(t_link_href)
                i+=1

https://www.ptt.cc/bbs/Beauty/M.1768138487.A.38D.html
[正妹] IVE 李瑞 Leeseo
******************************************************************************************
fp=M.1768138487.A.38D.html/meta.json
******************************************************************************************
https://i.meee.com.tw/NLKrusg.jpg
https://i.meee.com.tw/uk7jLzc.jpg
https://i.meee.com.tw/vm1gvw8.jpg
https://i.meee.com.tw/0uiBGz9.jpg
https://i.meee.com.tw/VqWhvrt.gif
https://i.meee.com.tw/kA8b1Gh.gif
https://i.meee.com.tw/Jem80dZ.gif
https://i.meee.com.tw/NW70WWZ.gif
******************************************************************************************
fp=./meta.json
******************************************************************************************
https://www.ptt.cc/bbs/Beauty/M.1768153595.A.CE3.html
[正妹] 陳奶糖
******************************************************************************************
fp=M.1768153595.A.CE3.html/meta.json
************************************************

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
get_page_data(url = "https://www.ptt.cc/bbs/Beauty/index4000.html")

https://www.ptt.cc/bbs/Beauty/M.1768090431.A.055.html
[正妹] 短髮妹子
yokann
1/11
https://i.urusai.cc/WOXGZ.jpg
https://i.urusai.cc/eqEVB.jpg
https://i.urusai.cc/trAQM.jpg
https://i.urusai.cc/felIL.jpg
https://i.urusai.cc/cxb79.jpg
https://i.urusai.cc/1YIPm.jpg
--------------------
https://www.ptt.cc/bbs/Beauty/M.1768090498.A.593.html
[正妹] 黑絲白襯衫
yokann
1/11
https://i.urusai.cc/B21qO.jpg
https://i.urusai.cc/udKj2.jpg
https://i.urusai.cc/m02TX.jpg
https://i.urusai.cc/mBelb.jpg
https://i.urusai.cc/NbpLc.jpg
https://i.urusai.cc/SUODz.jpg
https://i.urusai.cc/I7EiT.jpg
https://i.urusai.cc/WLk5q.jpg
https://i.urusai.cc/bEQv1.jpg
https://i.urusai.cc/g0UGI.jpg
https://i.urusai.cc/P38yj.jpg
https://i.urusai.cc/Uw1D0.jpg
https://i.urusai.cc/HztXH.jpg
https://i.urusai.cc/ys5ci.jpg
--------------------
https://www.ptt.cc/bbs/Beauty/M.1768095730.A.652.html
[正妹] Elyssa Barr佛羅里達直播主有大福利
FWMK2RC6RS
1/11
https://images2.imgbox.com/df/24/XasnkoCH_o.jpg
https://images2.imgbox.com/70/4b/hXCu0Kbc_o.jpg
https://ima

In [ ]:
# [{}, {}, {}]
import glob
glob.glob("./*/meta.json")
# glob.glob("./*/*.jpg")

['./M.1768138487.A.38D.html/meta.json', './M.1768153595.A.CE3.html/meta.json']

In [ ]:
import glob
import pandas as pd

table = []
for fp in glob.glob("./*/meta.json"):
    f = open(fp, "r", encoding="utf-8")
    content = f.read()
    f.close()
    row = json.loads(content)
    table.append(row)
# table
df = pd.DataFrame(table)
df

,uid,board,title,datetime,content,pushes
0,wafiea708 (元素708),Beauty,[正妹] IVE 李瑞 Leeseo,Sun Jan 11 21:34:44 2026,\nhttps://i.meee.com.tw/NLKrusg.jpg\n\nhttps:/...,"[{'tag': 1, 'uid': 'toolittle', 'text': '瑞寶 正..."
1,yokann (我怕練太壯),Beauty,[正妹] 陳奶糖,Mon Jan 12 01:46:33 2026,\nhttps://i.imgur.com/USsogRa.jpeg\nhttps://i....,[]


In [ ]:
# 多一欄分類catagory: 正妹
# datetime(format): 2026/01/12 09:20:08
def func(s):
    # s = "[正妹] 許雅涵"
    if "[" in s and "]" in s:
        return s.split("]")[0].replace("[", "")
    else:
        return "-"
df["category"] = df["title"].apply(func)
df

,uid,board,title,datetime,content,pushes,category
0,wafiea708 (元素708),Beauty,[正妹] IVE 李瑞 Leeseo,Sun Jan 11 21:34:44 2026,\nhttps://i.meee.com.tw/NLKrusg.jpg\n\nhttps:/...,"[{'tag': 1, 'uid': 'toolittle', 'text': '瑞寶 正...",正妹
1,yokann (我怕練太壯),Beauty,[正妹] 陳奶糖,Mon Jan 12 01:46:33 2026,\nhttps://i.imgur.com/USsogRa.jpeg\nhttps://i....,[],正妹


In [ ]:
# strftime 是將時間物件（如 datetime 或 time.struct_time）格式化成字串 (String from Time)，
# strptime 則是將 字串 解析成時間物件 (String Parse Time)。
# 簡單來說，strftime 負責「時間轉字串」以利顯示
# strptime 負責「字串轉時間」以便計算與操作。

import datetime as dt

def func(s):
    # s = "Mon Jan 12 01:50:24 2026"
    d = dt.datetime.strptime(s, "%a %b %d %H:%M:%S %Y")
    s2 = d.strftime("%Y/%m/%d %H:%M:%S")
    return s2

df["datetime"].apply(func)

,datetime
0,2026/01/11 21:34:44
1,2026/01/12 01:46:33


In [ ]:
import urllib.request
from urllib import request
request.urlopen
# !!!
from urllib.request import urlopen, urlretrieve
urlopen

In [ ]:
import os
import time
import urllib.request as req
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc


# 如果你是MAC, 遇到SSL Certificate Failed: 就加這兩行
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 你要把你所有的平常動作模擬在這裡
if __name__ == '__main__':
    driver = uc.Chrome(use_subprocess=False)
    driver.get("https://www.google.com")
    driver.maximize_window()

    time.sleep(3)

In [ ]:
import os
import time
import urllib.request as req
from os.path import dirname

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc


# 如果你是MAC, 遇到SSL Certificate Failed: 就加這兩行
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 你要把你所有的平常動作模擬在這裡
if __name__ == '__main__':
    driver = uc.Chrome(use_subprocess=False)
    driver.get("https://www.google.com")
    driver.maximize_window()
    # bs: find/find_all
    # selenium: find_element/find_elements
    # TAG_NAME/CLASS_NAME/ID
    e = driver.find_element(By.CLASS_NAME, "gLFyf")
    # click() send_keys()
    e.send_keys("chiikawa")
    e.send_keys(Keys.ENTER)

    time.sleep(3)
    es = driver.find_elements(By.CLASS_NAME, "C6AK7c")
    es[2].click()

    time.sleep(3)

    dn = "chiikawa"
    if not os.path.exists(dn):
        os.makedirs(dn)
    es = driver.find_elements(By.TAG_NAME, "img")
    for e in es:
        e_src = e.get_attribute("src")
        if not e_src == None:
            fn = dn + "/" + str(time.time()) + ".png"
            req.urlretrieve(e_src, fn)
    # bs: ["href"]/get_text() .text
    # selenium: get_attribute("href")/.text

    time.sleep(3)
    driver.quit()

In [ ]:
import os
import time
import urllib.request as req
from os.path import dirname

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc


# 如果你是MAC, 遇到SSL Certificate Failed: 就加這兩行
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 你要把你所有的平常動作模擬在這裡
# cookie: 你做完第一次, 伺服器要求你設置一組通關密碼在瀏覽器, 下次會被自動帶入以供檢查
if __name__ == '__main__':
    driver = uc.Chrome(use_subprocess=False)

    driver.get("https://www.google.com")
    driver.maximize_window()

    cookie_str = ""
    for c in cookie_str.split(";"):
        n, v = c.split("=", 1)
        n, v = n.strip(), v.strip()
        print(n, v)
        driver.add_cookie({"name": n, "value": v})

    # driver.refresh()
    driver.refresh()
    time.sleep(3)

    driver.quit()